In [11]:
filename="Basilashvili Rinderknech"
filename_csv=filename+".csv"
#correct the csv file generated by swing by comparing to the score
lst_players=filename_csv.split(".")[0].split(" ")
name1=lst_players[0]
name2=lst_players[1]
#set the time just before the first serve played
start=6*60+15

In [12]:
#COPY the 2nd sheet fo the xlsx file to a proper CSV file
import openpyxl
import csv
import pandas as pd
  
# open given workbook 
# and store in excel object 
wb = openpyxl.load_workbook(filename+".xlsx")
sheets = wb.sheetnames
sheet = wb[sheets[1]]  
  
# writer object is created
col = csv.writer(open(filename_csv,
                      'w', 
                      newline=""))
  
# writing the data in csv file
for r in sheet.rows:
    # row by row write 
    # operation is perform
    col.writerow([cell.value for cell in r])
  
# read the csv file and 
# convert into dataframe object 
df = pd.DataFrame(pd.read_csv(filename+".csv"))
  
# show the dataframe
df

Player  Shot             Type    Stroke     Spin  Speed (KM/H)  Point  \
0      Player     0             none      Feed     Flat     36.237053      1   
1    Opponent     1             none  Backhand  Topspin    106.635071      1   
2      Player     0             none  Backhand  Topspin     66.550903      2   
3      Player     0             none  Backhand  Topspin     62.534756      4   
4    Opponent     1            serve     Serve     Flat     44.957775      5   
..        ...   ...              ...       ...      ...           ...    ...   
563  Opponent     2           return  Forehand     Flat     68.356392    199   
564    Player     3   serve_plus_one  Forehand     Flat     95.847763    199   
565  Opponent     4  return_plus_one  Forehand  Topspin     84.989906    199   
566  Opponent     0             none  Backhand  Topspin     50.624168    201   
567    Player     1            serve     Serve     Flat    117.033859    202   

     Game  Set  Bounce Depth  ... Hit Zone Hit Side   Hit (x)    Hit (y)  \
0       0    0   service_box  ...    deuce     near  1.883357   2.058032   
1       0    0  no_mans_land  ...    deuce      far -0.881003  18.850418   
2       0    0           out  ...       ad      far  0.538222  19.376108   
3       0    0  no_mans_land  ...       ad     near -1.055682   0.579689   
4       0    0   service_box  ...    deuce      far  0.034740  23.147434   
..    ...  ...           ...  ...      ...      ...       ...        ...   
563     0    0  no_mans_land  ...       ad     near -1.112242  -0.154579   
564     0    0  no_mans_land  ...       ad      far  1.496063  24.268629   
565     0    0   service_box  ...    deuce     near  1.694612  -0.793481   
566     0    0  no_mans_land  ...       ad      far  0.182055  16.637714   
567     0    0  no_mans_land  ...       ad     near  0.208532   0.958335   

      Hit (z)      Direction Result  Favorited  Start Time   Video Time  
0    1.721165            ---     In      False    12:46:30    35.680000  
1    1.012932      inside in    Net      False    12:46:32    37.580002  
2    1.251543    cross court    Net      False    12:48:53   178.679993  
3    1.288213    cross court     In      False    12:50:34   279.279999  
4    1.782537     down the T     In      False    12:52:11   376.880005  
..        ...            ...    ...        ...         ...          ...  
563  0.917488     inside out     In      False    14:03:53  4678.220215  
564  1.540422      inside in     In      False    14:03:54  4679.819824  
565  0.778244  down the line    Net      False    14:03:55  4680.819824  
566  1.983566  down the line     In      False    14:06:03  4808.580078  
567  1.680203       out wide    Out      False    14:07:46  4911.379883  

[568 rows x 25 columns]

In [13]:
def get_all_shots_for_point(df):
    return (df[df["Point"].astype(int)==int(point_index)]).sort_values(by='Shot')

In [14]:
#REMOVE WARM UP shots and reindex points at 1
import pandas as pd
import numpy as np

df=pd.read_csv(filename_csv, index_col=0)
df=df.dropna(subset=["Point"])

try:
    df["Speed (KM/H)"]=round(df["Speed (KM/H)"])
except:
    pass
#remove the warm up
df=df[df["Video Time"] > start]
#reset the point at 1 (so we don't count warmup points anymore)
first_point=df.iloc[0]["Point"]
print("Point1 was actually: "+str(first_point))

#readjust point indexes
df["Point"]=df["Point"]-first_point+1
try:
    df = df.drop("Unnamed: 0", axis=1)
except:
    pass
df.to_csv(filename_csv)
df=pd.read_csv(filename_csv)
print("Point1 is now : "+str(first_point))

Point1 was actually: 5
Point1 is now : 5


In [5]:
#REMOVE ALL SHOTS where shot=0 
def remove_allshots_where_0(df):
    print("df before:{0}".format(len(df)))
    df=df[df["Shot"]>0]
    print("df after:{0}".format(len(df)))
    df.rename(columns = {'Speed (KM/H)':'Speed'," Hit (x)":"hit_x"," Hit (y)":"hit_y"," Hit (z)":"hit_z"}, inplace = True)
    dfP1=df[df.Player=="Player"]
    print("dfP1:{0}".format(len(dfP1)))
    dfP2=df[df.Player=="Opponent"]
    print("dfP2:{0}".format(len(dfP2)))
#remove_allshots_where_0(df)

In [15]:
#set the winner of the point for every shot

def set_point_winner(df, point_index: int):
    """
    Returns
    1 if player1 won the point
    0 if player 2
    -1 if player 1 hit his serve out
    -2 if player 2 hit his serve out
    """
    df_all_shots_for_point = get_all_shots_for_point(df)
    df_last_shot = df_all_shots_for_point[df_all_shots_for_point["Shot"]==df_all_shots_for_point["Shot"].max()]
    last_shot_result = df_last_shot.iloc[0]["Result"]
    index_winner = -1
    str_player_last_shot = df_last_shot.iloc[0]["Player"]
    print(str_player_last_shot)
    if (df_last_shot.iloc[0]["Shot"]==1) and (last_shot_result in ["Out", "Net"]):
        if (str_player_last_shot == "Player"):
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "0" #DF
            else:
                index_winner = "-1" #Player serve out or net
        else:
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "1"#DF
            else:
                index_winner = "-2" #Opponent serve out or net
    elif (str_player_last_shot == "Player"):
        if (last_shot_result in ["In"]):
            index_winner = "1"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "0"
        else:
            index_winner = "-1"
    elif (str_player_last_shot == "Opponent"):
        if (last_shot_result in ["In"]):
            index_winner = "0"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "1"
        else:
            index_winner = "-1"
    else:
        index_winner = "-1"
    #print(str(index_winner))
    df.loc[df["Point"]==point_index,"Winner_Point"]=index_winner
    
#set_point_winner(df, 8)

In [16]:
#FOR EACH UNIQUE POINT, set point_winner for all shots of this point
for point_index in df["Point"].unique():
    set_point_winner(df, point_index)

#SAVE FILE
df.to_csv(filename_csv)

Opponent
Opponent
Player
Player
Player
Opponent
Opponent
Player
Opponent
Opponent
Opponent
Player
Player
Opponent
Player
Opponent
Opponent
Opponent
Player
Player
Player
Player
Player
Opponent
Player
Player
Opponent
Opponent
Player
Opponent
Player
Player
Player
Opponent
Player
Opponent
Player
Opponent
Player
Player
Player
Opponent
Player
Player
Player
Player
Opponent
Player
Opponent
Player
Player
Player
Opponent
Player
Opponent
Opponent
Opponent
Opponent
Player
Player
Player
Player
Opponent
Player
Player
Opponent
Player
Opponent
Opponent
Player
Opponent
Opponent
Opponent
Player
Player
Player
Player
Player
Opponent
Opponent
Opponent
Opponent
Player
Player
Player
Player
Player
Player
Opponent
Player
Player
Player
Player
Player
Opponent
Opponent
Opponent
Player
Opponent
Opponent
Player
Opponent
Player
Player
Player
Player
Opponent
Opponent
Player
Opponent
Player
Player
Opponent
Player
Player
Player
Opponent
Player
Player
Opponent
Player
Player
Player
Player
Opponent
Player
Opponent
Opponen

In [17]:
print(name2 + " on SERVE, point winners:")
df[(df["Shot"]==1)&(df["Player"]=="Opponent")]["Winner_Point"].value_counts()

Rinderknech on SERVE, point winners:


0     43
1     22
-2    11
Name: Winner_Point, dtype: int64

In [18]:
print(name1 + " on SERVE, point winners:")
df[(df["Shot"]==1)&(df["Player"]=="Player")]["Winner_Point"].value_counts()

Basilashvili on SERVE, point winners:


1     38
0     32
-1    17
Name: Winner_Point, dtype: int64

In [10]:
"""
check server for each serve!
check serve (if out and nb shots > 2 then it s IN) 
synchronize time between video score and file score
for each point:
	compare point winner vs score file
	- find point scored at closest time in file (max -5, +8sec)
	- if found point and error point between both sources:
		if 1 shot only
			if result == out
				result=IN
		if 2+ shots
			correct last shot from losing player => in to OUT
			remove all next shots
	- if no error
		if some earlier shots have OUT, correct them as IN
	- if not found point (+10, -10) and shot1==out or net (only serve)
		if result==in or result==out
			correct to result=out
			remove all next shots
"""
def check_server(df, point_index, df_scores):
    """
    Check if the server is right
    If not, update all shots behind serve
    """
    df_all_shots_for_point = get_all_shots_for_point(df)
    row_serve = df_all_shots_for_point[str(df_all_shots_for_point['Shot'])=="1"]
    row_last_shot = df_all_shots_for_point[df_all_shots_for_point["Shot"]==df_all_shots_for_point["Shot"].max()]
    last_shot_result = row_last_shot.iloc[0]["Result"]
    index_winner = -1
    str_player_last_shot = row_last_shot.iloc[0]["Player"]
    #if ()
    return df

#FOR EACH UNIQUE POINT, set point_winner for all shots of this point
"""df_scores = pd.read_csv(filename_csv.replace("?csv","-scoreok.csv"))
for point_index in df["Point"].unique():
    check_server(df, point_index, df_scores)"""


'df_scores = pd.read_csv(filename_csv.replace("?csv","-scoreok.csv"))\nfor point_index in df["Point"].unique():\n    check_server(df, point_index, df_scores)'